In [16]:
#Load libraries and data
#import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor

#Configure Panda
pd.options.display.width = 200

train = pd.read_csv('data/train_v2.csv')
test = pd.read_csv('data/sample_submission_v2.csv')
transactions = pd.read_csv('data/transactions_v2.csv')
members = pd.read_csv('data/members_v3.csv')
print("hey")



hey


In [17]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


In [ ]:
#import current modified verion (see feature engineering file)
'final_members.csv'

In [70]:
#Prepare test data
transactions = transactions[['msno','transaction_date','membership_expire_date', 'is_cancel']]

#Match 'msno' in transactions and training
pred_columns_merged = pd.merge(transactions, train, on='msno')


#Remove duplicates of 'msno' in prediction columns
pred_columns_dropped = pred_columns_merged.drop_duplicates(['msno'], keep='first')


#Size pred_columns (samples) to equal y_train (lables)
pred_columns_dropped = pred_columns_dropped.iloc[:,:970960]
print(pred_columns_dropped.head())


#Remove 'msno' so pred_columns_train has only numeric values
pred_columns_train = pred_columns_dropped.iloc[:,1:4]
print(pred_columns_train.head())

#Prediction columns for testing
p_columns_merged = pd.merge(transactions, test, on='msno')

p_columns_dropped = p_columns_merged.drop_duplicates(['msno'], keep='first')
print(p_columns_dropped.head())


#Remove msno to have only numeric values
pred_columns_test = p_columns_dropped.iloc[:,1:4]
print(pred_columns_test.head())

#print(pred_columns_train.count())
#print(pred_columns_test.count())

                                            msno  transaction_date  membership_expire_date  is_cancel  is_churn
0   ++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=          20150809                20190412          0         0
39  +/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=          20170303                20170422          0         0
40  +/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=          20170329                20170331          1         1
41  +00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=          20170323                20170423          0         0
42  +0KcMm8JNCW08lTp3Lyz5Ger/47u3yj9H2xLf8lyAj8=          20151112                20180613          0         0
    transaction_date  membership_expire_date  is_cancel
0           20150809                20190412          0
39          20170303                20170422          0
40          20170329                20170331          1
41          20170323                20170423          0
42          20151112                20180613    

In [62]:
#Split data into train and test sets

X_train = pred_columns_train
#y_train = train.is_churn.iloc[:933578]
y_train = pred_columns_dropped.is_churn

X_test = pred_columns_test
#y_test = test.is_churn.iloc[:907470]
y_test = p_columns_dropped.is_churn

print("Size of X_train: ")
print(X_train.count())

print("Size of y_train: ")
print(y_train.count())

print("Size of y_test: ")
print(y_test.count())

print("Size of X_test:")
print(X_test.count())

# fit model, no training data
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Make predicitons for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

#Evaluate predictions

accuracy = accuracy_score(y_test, predictions)

print("Accuracy: %.2f%%" % (accuracy * 100.0))



Size of X_train: 
transaction_date          933578
membership_expire_date    933578
is_cancel                 933578
dtype: int64
Size of y_train: 
933578
Size of y_test: 
907470
Size of X_test:
transaction_date          907470
membership_expire_date    907470
is_cancel                 907470
dtype: int64
Accuracy: 94.97%


In [63]:
print("Size of X_train: ")
print(X_train.size)

print("Size of y_train: ")
print(y_train.size)

print("Size of y_pred: ")
print(y_pred.size)

print("Size of X_test:")
print(X_test.size)

Size of X_train: 
2800734
Size of y_train: 
933578
Size of y_pred: 
907470
Size of X_test:
2722410


In [69]:
#Prepare submission file
msno = p_columns_dropped.msno #PROBLEM: we cant submit anymore!
    
my_submission = pd.DataFrame({'msno': msno, 'is_churn': y_pred})
cols = my_submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
my_submission =my_submission[cols]
print(my_submission.head())
my_submission.count()

#my_submission.to_csv('submission.csv', index=False)


                                            msno  is_churn
0   ++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=  0.000000
39  +/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=  0.095261
40  +00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=  0.007255
41  +0KcMm8JNCW08lTp3Lyz5Ger/47u3yj9H2xLf8lyAj8=  0.300000
60  +0MeUJe1cGb4O97gJoTPUPjiONQzi9BmxDuKRZn+E2o=  0.039038
